In [7]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from dateutil import parser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import codecs
import json
import datetime
import os
from urllib.request import urlretrieve
account = "testToPage"
password = "selenium Cookie"

In [8]:
def gamePage(pages,game_index):
    
    game_index = game_index*50
    names = [] #陣列拿來存取資料
    tags = []
    introductions = []
    currentReviews = []
    allReviews = []
    dates = []
    developers = []
    companys = []
    texts = []
    prices = []
    count = 0
    for page in pages:
        tag = ""
        options = webdriver.ChromeOptions()
        options.add_argument('lang=zh_TW.UTF-8')              #設定成繁體中文
        driver = webdriver.Chrome('chromedriver',options =options)
        driver.get(page)
        time.sleep(5)
        
        if("error_box" in driver.page_source): #若需要登入
            loginPage = driver.find_element_by_class_name("global_action_link")#點擊登入
            loginPage.click()
            accountBar = driver.find_element_by_id("input_username")           #輸入帳號密碼    
            accountBar.send_keys(account)
            passwordBar = driver.find_element_by_id("input_password")
            passwordBar.send_keys(password)
            passwordBar.send_keys(Keys.ENTER)
            time.sleep(5)
            goIn = driver.find_element_by_css_selector("[class='btnv6_blue_hoverfade btn_medium']")#點擊檢視頁面(class名稱有空白)
            goIn.click()
        
        if("agegate_wizard"in driver.page_source): 
            if("ageYear" in driver.page_source):
                selectorYear = Select(driver.find_element_by_id("ageYear"))
                selectorYear.select_by_value("1995")
            goIn = driver.find_element_by_css_selector("[class='btnv6_blue_hoverfade btn_medium']")#點擊檢視頁面(class名稱有空白)
            goIn.click()
            time.sleep(5)
        
        soup = bs(driver.page_source,"html.parser")
        
        name = soup.find('div',{'class':'apphub_AppName'}).text.strip()#抓遊戲名稱
        if(name not in names):
            
            if('game_area_purchase_game_wrapper' in driver.page_source):
                priceBox = soup.find('div',{'class':'game_area_purchase_game_wrapper'})#原價的位置
            else:
                priceBox = soup.find('div',{'class':'game_area_purchase_game '})#免費的位置
    
            price = priceBox.find('div',{'class':'game_purchase_price price'})#抓價錢
            if price !=None:
                price = price.text.strip()
            else:
                price = priceBox.find('div',{'class':'discount_final_price'})
                if price !=None:
                    price = price.text.strip()
                else:
                    price = '免費遊玩'
        
            introduction = soup.find('div',{'class':'game_description_snippet'}) #抓介紹文章且判斷是否存在
            if introduction != None:
                introduction = introduction.text.strip()
            
            Container = soup.find('div',{'class':'user_reviews'}) #抓評論和發表時間、開發者的區間
            reviewrow = Container.find_all('div',{'class':'user_reviews_summary_row'})
        
            if(len(reviewrow)>1):                            #判斷評論區長度 1表示只有所有評論
                currentReview = reviewrow[0].find('span').text.strip()
                allReview = reviewrow[1].find('span')
                if(allReview!= None): 
                    allReview = allReview.text.strip()
            else:
                currentReview = None
                allReview = reviewrow[0].find('span')
                if(allReview!= None):                        #判斷所有評論存不存在
                    allReview = allReview.text.strip()
            
            date = Container.find('div',{'class':'date'})  #抓日期
            if date != None:
                date = date.text.strip()
        
            developer = Container.find('div',{'id':'developers_list'}).text.strip()#抓開發者
        
            companyRow = Container.find_all('div',{'class':'dev_row'}) #抓是否有廠商
            if(len(companyRow)>1):
                company = companyRow[1].find('div',{'class':'summary column'}).text.strip()
            else:
                company = None
            
            tagsAndLink = soup.find('div',{'class':'glance_tags popular_tags'})
            tagcount = 0;
            for reader in tagsAndLink.find_all('a', href=True):
                tagcount+=1
                if(reader.text.strip() not in tag):
                    if(tagcount<len(tagsAndLink.find_all('a'))):
                        tag = tag+reader.text.strip()+","
                    else:
                        tag = tag+reader.text.strip()
                    
            img = soup.find('img',{'class':'game_header_image_full'})['src']# 抓圖片連結
            local = os.path.join('D:\\1\\%s.jpg' % game_index)# 設定下載位址
            urlretrieve(img,local)
            game_index+=1
            
            names.append(name) #將資料丟入陣列
            tags.append(tag)
            introductions.append(introduction)
            currentReviews.append(currentReview)
            allReviews.append(allReview)
            dates.append(date)
            developers.append(developer)
            companys.append(company)
            prices.append(price)
            
            count+=1
            print(count)
        

        time.sleep(60)
        driver.close()
    for i in range(0,len(names)):
        text = {"遊戲名稱":names[i],"價錢":prices[i],"遊戲介紹":introductions[i],"最近評論":currentReviews[i]
                ,"所有評論":allReviews[i],"發行日期":dates[i],"開發者":developers[i],"公司":companys[i],
                "標籤":tags[i],"連結":pages[i]}
        texts.append(text)
    jsons = json.dumps(texts,ensure_ascii=False)
    today = datetime.datetime.today().strftime("%m%d").__str__()
    fileName = "Steam"+today+'.txt'
    fp = codecs.open(fileName, 'w', 'utf-8')
    fp.write(jsons)
    fp.close()

In [9]:
def getSerchPage():
    df=pd.read_csv('serchPage.csv',encoding ="utf_8_sig")
    links = df['連結'].values
    return links

In [11]:
if __name__ == '__main__':
    print(datetime.datetime.now())
    links = getSerchPage()      #需先跑過搜尋的程式
    gamePage(links,1)       #設置要爬的數量
    print(datetime.datetime.now())

2018-12-12 00:40:20.952860
1
2018-12-12 00:42:44.252553
